Insert cells programmaticaly to python notebooks

In [4]:
import os
from glob import glob
import json
import numpy as np

In [2]:
files = glob("*.ipynb")

In [5]:
files = [fn for fn in glob('*') 
         if not os.path.basename(fn).endswith('done.ipynb')]

In [74]:
files_to_overwrite = files[6:]
files_to_overwrite

['cli.027.ipynb',
 'dis.004.ipynb',
 'dis.005.ipynb',
 'dis.007.ipynb',
 'dis.008.ipynb',
 'dis.012.ipynb',
 'ene.003.1.ipynb',
 'ene.003.2.ipynb',
 'ene.004.ipynb',
 'ene.006.ipynb',
 'ene.015.ipynb',
 'ene.018.ipynb',
 'foo.005.ipynb',
 'foo.017.ipynb',
 'foo.023.ipynb',
 'foo.028.ipynb',
 'soc.031.ipynb',
 'soc.032.ipynb',
 'wat.015.ipynb',
 'wat.029.ipynb']

In [87]:
standard_fare = [
    
{'cell_type': 'markdown',
  'metadata': {},
  'source': [
  'Import libraries']},
{'cell_type': 'code',
  'execution_count': 0,
  'metadata': {'collapsed': True},
  'outputs': [],
  'source': [
"""# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob"""]},
    
    
    
    
{'cell_type': 'markdown',
  'metadata': {},
  'source': [
  's3 tools']},
{'cell_type': 'code',
  'execution_count': 0,
  'metadata': {'collapsed': False},
  'outputs': [],
  'source': [
  
"""s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/*"

s3_file = "*.tif"

s3_key_orig = s3_folder + s3_file
s3_key_edit = s3_key_orig[0:-4] + "_edit.tif"


class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()""" ]},
   
    
    
{'cell_type': 'markdown',
  'metadata': {},
  'source': [
  'Potentially useful for online directories']},   
    
    {'cell_type': 'code',
  'execution_count': 0,
  'metadata': {'collapsed': False},
  'outputs': [],
  'source': [
"""# View files in a ftp, see cli.015
remote_path = "ftp://cidportal.jrc.ec.europa.eu/jrc-opendata/EDGAR/datasets/v431_v2"
remote_path_BC = remote_path + "/BC/TOTALS/"
file = req.urlopen(remote_path_BC).read().splitlines()

# Copy from https or ftp
with(closing(urlopen(online_folder + most_recent))) as r:
    with(open(local_orig, 'wb')) as f:
        shutil.copyfileobj(r, f)"""     
  ]},
    
    
{'cell_type': 'markdown',
  'metadata': {},
  'source': [
  'Define local file locations']},             
{'cell_type': 'code',
  'execution_count': 0,
  'metadata': {'collapsed': False},
  'outputs': [],
  'source': [
"""local_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/*"
file_name = "*"]

local_orig = local_folder + file_name

orig_extension_length = 4 #4 for each char in .tif
local_edit = local_orig[:-orig_extension_length] + "edit.tif" """     
  ]},
    
    
    
        
    
{'cell_type': 'markdown',
  'metadata': {},
  'source': ['Use rasterio to reproject and compress']},
{'cell_type': 'code',
  'execution_count': 0,
  'metadata': {'collapsed': False},
  'outputs': [],
  'source': [

"""# Note - this is the core of Vizz's netcdf2tif function

with rio.open(local_orig, 'r') as src:
    # This assumes data is readable by rasterio
    # May need to open instead with netcdf4.Dataset, for example
   
    data = src.read()[0]
    
    rows = data.shape[0]
    columns = data.shape[1]
    
    print(rows)
    print(columns)
    
    # Latitude bounds
    south_lat = -90
    north_lat = 90

    # Longitude bounds
    west_lon = -180
    east_lon = 180
    
    transform = rasterio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)
    
    # Profile
    
    no_data_val = *
    target_projection = 'EPSG:4326'
    target_data_type = np.float64
    
    profile = {
        'driver':'GTiff', 
        'height':rows, 
        'width':columns, 
        'count':1, 
        'dtype':target_data_type, 
        'crs':target_projection, 
        'transform':transform, 
        'compress':'lzw', 
        'nodata': no_data_val
    }
    
    with rio.open(local_edit, "w", **profile) as dst:
        dst.write(data.astype(profile["dtype"]), 1)"""
  ]},

    

 {'cell_type': 'markdown',
  'metadata': {},
  'source': ['Upload orig and edit files to s3']},
 {'cell_type': 'code',
  'execution_count': 0,
  'metadata': {'collapsed': False},
  'outputs': [],
  'source': [
      
"""# Original
s3_upload.upload_file(local_orig, s3_bucket, s3_key_orig,
                         Callback=ProgressPercentage(local_orig))

# Edit
s3_upload.upload_file(local_edit, s3_bucket, s3_key_edit,
                         Callback=ProgressPercentage(local_edit))"""
      
  ]}
]

In [88]:
s_fare = standard_fare
for cell in s_fare:
    if 'execution_count' in cell.keys():
        cell['execution_count'] = 0

In [81]:
s_fare

[{'cell_type': 'markdown', 'metadata': {}, 'source': ['Import libraries']},
 {'cell_type': 'code',
  'execution_count': 0,
  'metadata': {'collapsed': True},
  'outputs': [],
  'source': ['# Libraries for downloading data from remote server (may be ftp)\nimport requests\nfrom urllib.request import urlopen\nfrom contextlib import closing\nimport shutil\n\n# Library for uploading/downloading data to/from S3\nimport boto3\n\n# Libraries for handling data\nimport rasterio as rio\nimport numpy as np\n# from netCDF4 import Dataset\n# import pandas as pd\n# import scipy\n\n# Libraries for various helper functions\n# from datetime import datetime\nimport os\nimport threading\nimport sys\nfrom glob import glob']},
 {'cell_type': 'markdown', 'metadata': {}, 'source': ['s3 tools']},
 {'cell_type': 'code',
  'execution_count': 0,
  'metadata': {'collapsed': False},
  'outputs': [],
  'source': ['s3_upload = boto3.client("s3")\ns3_download = boto3.resource("s3")\n\ns3_bucket = "wri-public-data"\ns3

In [70]:
f = open(files_to_overwrite[3], 'r')
j = json.load(f)

In [71]:
j["cells"] = [j["cells"][0]]

In [72]:
for fare in standard_fare:
    j["cells"].append(fare)

In [73]:
with open('test.ipynb', 'w') as outfile:
    json.dump(j, outfile)

In [82]:
files_to_overwrite

['cli.027.ipynb',
 'dis.004.ipynb',
 'dis.005.ipynb',
 'dis.007.ipynb',
 'dis.008.ipynb',
 'dis.012.ipynb',
 'ene.003.1.ipynb',
 'ene.003.2.ipynb',
 'ene.004.ipynb',
 'ene.006.ipynb',
 'ene.015.ipynb',
 'ene.018.ipynb',
 'foo.005.ipynb',
 'foo.017.ipynb',
 'foo.023.ipynb',
 'foo.028.ipynb',
 'soc.031.ipynb',
 'soc.032.ipynb',
 'wat.015.ipynb',
 'wat.029.ipynb']

In [89]:
for file in files_to_overwrite:
    print(file)
    f = open(file, 'r')
    j = json.load(f)
    j["cells"] = [j["cells"][0]]
    for fare in s_fare:
        j["cells"].append(fare)
    with open(file, 'w') as outfile:
        json.dump(j, outfile)

cli.027.ipynb
dis.004.ipynb
dis.005.ipynb
dis.007.ipynb
dis.008.ipynb
dis.012.ipynb
ene.003.1.ipynb
ene.003.2.ipynb
ene.004.ipynb
ene.006.ipynb
ene.015.ipynb
ene.018.ipynb
foo.005.ipynb
foo.017.ipynb
foo.023.ipynb
foo.028.ipynb
soc.031.ipynb
soc.032.ipynb
wat.015.ipynb
wat.029.ipynb
